<a href="https://colab.research.google.com/github/calbarber21/Sentiment-Analysis-Thesis/blob/master/CleanThesisTxt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
from io import StringIO
import os
import re
import numpy as np
from linecache import getline
import time
import datetime
import sys
import pandas as pd 
import pdb
!pip install xlsxwriter
import xlsxwriter
import nltk
nltk.download('punkt')
from nltk import tokenize
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
#google collab specific line to connect to google drive
from google.colab import drive 
drive.mount('/content/gdrive')
#change directoy to .txt files directory
%cd /content/gdrive/My\ Drive/PDF Data Thesis/Text Data

     |████████████████████████████████| 153kB 2.0MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFQ8QNU2vNqHY9ZV0aqx2xhyZ6c0UwHaGEhM8p_f-8odExyb5DxUhQ
Mounted at /content/gdrive
/content/gdrive/My Drive/PDF Data Thesis/Text Data


Parse Corpus

In [ ]:
files=[i for i in os.listdir() if i.endswith('txt')]
data = pd.DataFrame({'Title':[],'Date':[],'article':[]})
for file in files :
  fileX= open(file, "r")
  text = fileX.read()
  text = text.split("End of Document")
  count = 0
  for episode in text :
    if count > 0 :
      newrow={'Title':title, 'Date':date,'article':body}
      data=data.append(newrow, ignore_index=True)
    getLines = False
    body = []
    episode = episode.splitlines()
    episode = [episode for episode in episode if episode.strip() != '']
    for i , line in enumerate(episode,start=0) :
      line = line.strip()
      if ("The New York Times" == line) : 
        if (i == 2) :
          title = episode[i - 2].strip() + " " + episode[i - 1].strip()
        else :
          title = episode[i - 1].strip()
        date = re.findall(r'(?:\d{2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}, )?\d{4}',episode[i + 1])
      if line.startswith("Body") :
        getLines = True
      if getLines :
        body.append(line)
    count = count + 1

                                                   Title  ...                                            article
0                           Botching the Great Recession  ...  [Body, It has been ten years since the failure...
1      Rebuffed by Trump on Tariffs, Businesses Mount...  ...  [Body, WASHINGTON — They’ve testified, tweeted...
2                   'Anonymous' Is Hiding in Plain Sight  ...  [Body, More and more, I wonder if the disgrunt...
3      Economic Bellwether Is Making a Midwestern Cit...  ...  [Body, ELKHART, Ind. -- The tables were filled...
4                       Paid Notice: Deaths BOVIN, HENRY  ...  [Body, BOVIN--Henry, 97, of Morristown, NJ die...
...                                                  ...  ...                                                ...
20694       No Region Safe As Cases Soar, Fauci Cautions  ...  [Body, The government's top infectious disease...
20695          Optimism By Mnuchin Is Tempered By Powell  ...  [Body, The Treasury secretary and

Expand Contractions

In [ ]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


data_text = data[['article']]
data_text['article'] = data_text['article'].apply(', '.join)

documents = data_text
processed_docs = documents['article'].map(expand_contractions)
print(processed_docs)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0        Body, It has been ten years since the failure ...
1        Body, WASHINGTON — They’ve testified, tweeted,...
2        Body, More and more, I wonder if the disgruntl...
3        Body, ELKHART, Ind. -- The tables were filled ...
4        Body, BOVIN--Henry, 97, of Morristown, NJ died...
                               ...                        
20694    Body, The governments top infectious disease e...
20695    Body, The Treasury secretary and the Fed chair...
20696    Body, A small staff of advisers and other trus...
20697    Body, Borders and businesses are reopening, ev...
20698    Body, Tame markets do not mean the path back t...
Name: article, Length: 20699, dtype: object


Remove Accented Characters

In [ ]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

df = pd.DataFrame(processed_docs)
data_text = df[['article']]
#data_text['article'] = data_text['article'].apply(', '.join)

documents = data_text
processed_docs = documents['article'].map(remove_accented_chars)
print(processed_docs)

Remove Special Characters

In [ ]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

df = pd.DataFrame(processed_docs)
data_text = df[['article']]
#data_text['article'] = data_text['article'].apply(', '.join)

documents = data_text
processed_docs = documents['article'].map(remove_special_characters)
print(processed_docs)

Lemmatize

In [ ]:
import gensim
import unicodedata
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')


def lemmatize_stemming(text):
      return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
      result = []
      for token in gensim.utils.simple_preprocess(text):
          if token not in stopword_list and len(token) > 2 and token != "no" and token != "not":
              result.append(lemmatize_stemming(token))
      return result

df = pd.DataFrame(processed_docs)
data_text = df[['article']]
data_text['index'] = data_text.index
documents = data_text
processed_docs = documents['article'].map(preprocess)
processed_docs = processed_docs.map(remove_accented_chars)
processed_docs.to_csv('AllData_PreWeighted.csv')